In [2]:
import cvxpy as cp
import numpy as np

# Dados do problema
dias = ['Segunda', 'Terça', 'Quarta', 'Quinta']
disciplinas = ['Matemática', 'Português', 'Química', 'Física']
turmas = ['Turma A', 'Turma B']
blocos = ['Teoria (19h-21h)', 'Exercícios (21h-23h)']

# Recursos humanos por disciplina
prof_teoria = {
    'Matemática': 2,
    'Português': 2,
    'Química': 2,
    'Física': 2
}

prof_exercicios = {
    'Matemática': 2,
    'Português': 2,
    'Química': 2,
    'Física': 2
}

monitores = {
    'Matemática': 4,
    'Português': 2,  # Menos recursos
    'Química': 4,
    'Física': 4
}

# Variáveis de decisão
# Alocação de disciplina para cada turma em cada dia
X = cp.Variable((len(turmas), len(dias)), boolean=True)

# Alocação de professores de teoria
Y = cp.Variable((len(turmas), len(dias), len(disciplinas)), boolean=True)

# Alocação de professores de exercícios
Z = cp.Variable((len(turmas), len(dias), len(disciplinas)), boolean=True)

# Alocação de monitores
W = cp.Variable((len(turmas), len(dias), len(disciplinas)), integer=True)

# Restrições
constraints = []

# 1. Cada turma tem uma disciplina por dia
for t in range(len(turmas)):
    for d in range(len(dias)):
        constraints.append(cp.sum(X[t,d]) == 1)

# 2. Cada disciplina deve ser ministrada uma vez por turma
for t in range(len(turmas)):
    for disc in range(len(disciplinas)):
        constraints.append(cp.sum([X[t,d] for d in range(len(dias)) if disciplinas[disc] == 'Português']) == 1)

# 3. Português tem aulas unificadas
for d in range(len(dias)):
    constraints.append(X[0,d] == X[1,d])  # Mesma disciplina no mesmo dia para as duas turmas

# 4. Alocação de professores de teoria
for disc in range(len(disciplinas)):
    total = cp.sum(Y[:,:,disc])
    constraints.append(total <= prof_teoria[disciplinas[disc]])

# 5. Alocação de professores de exercícios
for disc in range(len(disciplinas)):
    total = cp.sum(Z[:,:,disc])
    constraints.append(total <= prof_exercicios[disciplinas[disc]])

# 6. Alocação de monitores
for disc in range(len(disciplinas)):
    for t in range(len(turmas)):
        for d in range(len(dias)):
            constraints.append(W[t,d,disc] >= 0)
            constraints.append(W[t,d,disc] <= monitores[disciplinas[disc]])

# 7. Distribuição equilibrada de monitores
for disc in range(len(disciplinas)):
    if disciplinas[disc] != 'Português':
        for t in range(len(turmas)):
            constraints.append(cp.sum(W[t,:,disc]) >= monitores[disciplinas[disc]] // 2 - 1)
            constraints.append(cp.sum(W[t,:,disc]) <= monitores[disciplinas[disc]] // 2 + 1)

# Função objetivo: Minimizar o número total de voluntários alocados
objective = cp.Minimize(
    cp.sum(Y) + cp.sum(Z) + cp.sum(W)
)

# Resolver o problema
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

# Exibir resultados
if problem.status == 'optimal':
    print("Alocação ótima encontrada!")
    print(f"Número total de voluntários alocados: {problem.value:.0f}")
    
    for t in range(len(turmas)):
        print(f"\n{turmas[t]}:")
        for d in range(len(dias)):
            disc_idx = np.argmax(X[t,d].value)
            print(f"{dias[d]}: {disciplinas[disc_idx]}")
            print(f"  Teoria: {Y[t,d,disc_idx].value:.0f} professor(es)")
            print(f"  Exercícios: {Z[t,d,disc_idx].value:.0f} professor(es)")
            print(f"  Monitores: {W[t,d,disc_idx].value:.0f}")
else:
    print("Não foi possível encontrar uma alocação válida.")

Não foi possível encontrar uma alocação válida.


/home/pedro.trindade/anaconda3/envs/optmization/lib/python3.13/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(
